In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv(override=True, dotenv_path=".env")

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

In [ ]:
from openai import OpenAI
from pydantic import BaseModel
import traceback

openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


class DreamInput(BaseModel):
    description: str
    temperature: float = 0.5
    model: str = "gpt-4o"


class DreamInterpretation(BaseModel):
    dream_interpretation: str
    emotions: list[str]
    suppressed_fears: list[str]
    suppressed_desires: list[str]
    practical_tips: list[str]


def analyze_dream(dream_input: DreamInput) -> DreamInterpretation:
    """
    Analyzes the dream and returns the interpretation.

    Args:
        dream_input (DreamInput): The dream input.

    Returns:
        DreamInterpretation: The dream interpretation.
    """
    try:
        system_prompt = """
            You are an advanced Dream Journal Analyzer AI that specializes in interpreting user-submitted dreams. Your role is to:
            Interpret Dreams: Analyze the dream’s symbols, scenarios, and emotions to provide meaningful insights. Base your analysis on common dream interpretation themes and symbolic meanings across cultures and psychology.
            Identify Patterns: Track and identify recurring themes or symbols over time, providing users with insights into potential subconscious patterns.
            Provide Contextual Suggestions: Offer suggestions to help users reflect on emotions or life events that may relate to their dreams.
            Be empathetic and thoughtful in your responses, ensuring users feel supported and validated while engaging with the app. Aim to provide balanced interpretations, neither overly literal nor purely abstract.
        """

        user_prompt = f"""
            Dream Description: {dream_input.description}
        """

        response = openai.beta.chat.completions.parse(
            model=dream_input.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {
                    "role": "user",
                    "content": user_prompt,
                },
            ],
            temperature=dream_input.temperature,
            response_format=DreamInterpretation,
        )
        return response.choices[0].message.parsed
    except Exception as e:
        print(f"An error occurred when interpreting the dream: {e}")
        traceback.print_exc()
        return None


def dream_interpretation_markdown(dream_interpretation: DreamInterpretation) -> str:
    """
    Formats the dream interpretation as markdown.

    Args:
        dream_interpretation (DreamInterpretation): The dream interpretation.

    Returns:
        str: The markdown formatted dream interpretation.
    """
    markdown = f"""
    ## Dream Interpretation
    
    **Interpretation:** {dream_interpretation.dream_interpretation}
    
    **Emotions:** {', '.join(dream_interpretation.emotions)}
    
    **Suppressed Fears:** {', '.join(dream_interpretation.suppressed_fears)}
    
    **Suppressed Desires:** {', '.join(dream_interpretation.suppressed_desires)}
    
    **Practical Tips:** {', '.join(dream_interpretation.practical_tips)}
    """
    return markdown

In [ ]:
from IPython.display import Markdown, display

dream_description = f"""
I was in a dream where Angelina Jolie was my girlfriend and we went for a walk in the park.
I was telling her about Programming and she was very interested in learning it.
So we sat on a bench and I started teaching her Python.
She was a quick learner and she was very happy to learn it.
"""

dream_input = DreamInput(description=dream_description)

dream_interpretation = analyze_dream(dream_input)

markdown = dream_interpretation_markdown(dream_interpretation)

display(Markdown(markdown))